In [1]:
# Some minor setup for the notebook
import warnings
import nest_asyncio

# Ignore all warnings
warnings.filterwarnings("ignore")

# Allows for running async code in Jupyter notebooks
nest_asyncio.apply()

In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
from datasets import load_dataset

ms_marco = load_dataset("microsoft/ms_marco", "v2.1")

In [33]:
import numpy as np
from sklearn.metrics import precision_score, recall_score


def ndcg(retrieved_list, relevant_list, k=None):
    def dcg(scores):
        return sum(score / (i + 1) for i, score in enumerate(scores))

    if k is None:
        k = len(retrieved_list)

    retrieved_scores = [1 if doc in relevant_list else 0 for doc in retrieved_list[:k]]
    ideal_scores = sorted(retrieved_scores, reverse=True)

    dcg_value = dcg(retrieved_scores)
    idcg_value = dcg(ideal_scores)

    return dcg_value / idcg_value if idcg_value > 0 else 0


def mean_reciprocal_rank(retrieved_list, relevant_list):
    ranks = []
    for relevant in relevant_list:
        try:
            rank = retrieved_list.index(relevant) + 1
        except ValueError:
            rank = len(retrieved_list) + 1  # Not found
        ranks.append(rank)
    return 1.0 / min(ranks) if ranks else 0.0


def calculate_ndcg(retrieved_list, relevant_list, k=None):
    def dcg(scores):
        return sum(score / (np.log2(idx + 2)) for idx, score in enumerate(scores))

    if k is None:
        k = len(retrieved_list)

    # Relevance scores for retrieved documents
    retrieved_scores = [1 if doc in relevant_list else 0 for doc in retrieved_list[:k]]
    # Ideal relevance scores
    ideal_scores = sorted(retrieved_scores, reverse=True)

    dcg_value = dcg(retrieved_scores)
    idcg_value = dcg(ideal_scores)

    return dcg_value / idcg_value if idcg_value > 0 else 0


def calculate_precision(retrieved_list, relevant_list):
    y_true = [1 if doc in relevant_list else 0 for doc in retrieved_list]
    y_pred = [1] * len(y_true)  # Assuming all retrieved documents are relevant for calculating precision
    return precision_score(y_true, y_pred, zero_division=0)


def calculate_recall(retrieved_list, relevant_list):
    y_true = [1 if doc in relevant_list else 0 for doc in relevant_list]
    y_pred = [1 if doc in retrieved_list else 0 for doc in relevant_list]
    return recall_score(y_true, y_pred, zero_division=0)


def compute_all_stats(retrieved_list, relevant_list):
    return {
        'MRR': mean_reciprocal_rank(retrieved_list, relevant_list),
        'nDCG': calculate_ndcg(retrieved_list, relevant_list, k=len(relevant_list)),
        'Precision': calculate_precision(retrieved_list, relevant_list),
        'Recall': calculate_recall(retrieved_list, relevant_list),
    }

In [5]:
# Use subset for demonstration
subset = ms_marco['train'].select(range(1000))

In [6]:
print(subset[2]['query'])

why did stalin want control of eastern europe


In [7]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

In [8]:
# Prepare the documents for BM25
bm25_docs = [Document(page_content=passage) for data in subset for passage in data['passages']['passage_text']]

# Initialize BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(bm25_docs)

In [17]:
import os
from ragstack_colbert import CassandraDatabase, ColbertEmbeddingModel

keyspace = "benchmarksmarco1000parallel"
database_id = os.getenv("ASTRA_DATABASE_ID")
astra_token = os.getenv("ASTRA_TOKEN")

database = CassandraDatabase.from_astra(
    astra_token=astra_token,
    database_id=database_id,
    keyspace=keyspace
)

embedding_model = ColbertEmbeddingModel()

[Jun 26, 10:12:08] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


In [34]:
from ragstack_langchain.colbert import ColbertVectorStore as LangchainColbertVectorStore

lc_vector_store = LangchainColbertVectorStore(
    database=database,
    embedding_model=embedding_model,
)

In [24]:
all_texts = []
all_metadatas = []
i = 0
for row in subset:
    all_texts.extend(row['passages']['passage_text'])
    all_metadatas.extend([{'row_id': i} for _ in row['passages']['is_selected']])
    i += 1


In [53]:
print("Number of texts:", len(all_texts))

Number of texts: 9992


In [ ]:
lc_vector_store.add_texts(all_texts, metadatas=all_metadatas)

In [57]:
from tqdm import tqdm

colbert_retrieved_list = []
colbert_relevant_list = []
for row in tqdm(subset):
    query = row['query']
    n_results = len(row['passages']['is_selected'])
    raw_results = lc_vector_store.similarity_search(query, k=n_results)
    retrieved_list = [result.page_content for result in raw_results]
    relevant_list = row['passages']['passage_text']

    colbert_retrieved_list.append(retrieved_list)
    colbert_relevant_list.append(relevant_list)

100%|██████████| 1000/1000 [11:07<00:00,  1.50it/s]


In [58]:
# calculate stats
colbert_stats = [compute_all_stats(retrieved_list, relevant_list) for retrieved_list, relevant_list in
                 zip(colbert_retrieved_list, colbert_relevant_list)]

In [55]:
# Now test BM25
bm25_retrieved_list = []
bm25_relevant_list = []

for row in tqdm(subset):
    query = row['query']
    n_results = len(row['passages']['is_selected'])
    raw_results = bm25_retriever.invoke(query, k=n_results)
    retrieved_list = [result.page_content for result in raw_results]
    relevant_list = row['passages']['passage_text']

    bm25_retrieved_list.append(retrieved_list)
    bm25_relevant_list.append(relevant_list)


100%|██████████| 1000/1000 [00:06<00:00, 155.95it/s]


In [56]:
bm25_stats = [compute_all_stats(retrieved_list, relevant_list) for retrieved_list, relevant_list in
              zip(bm25_retrieved_list, bm25_relevant_list)]

In [60]:
# Delete Kendall-Tau from all stats
for stats in colbert_stats + bm25_stats:
    del stats['Kendall-Tau']

In [ ]:
# Save the stats to a json file
import json

cwd = os.getcwd()

with open(f"{cwd}/colbert_bm25_stats.json", "w") as f:
    json.dump({"ColBERT": colbert_stats, "BM25": bm25_stats}, f)
    
print("Stats saved to colbert_bm25_stats.json")

In [11]:
# Load the stats from the json file
import json
import os
cwd = os.getcwd()
with open(f"{cwd}/colbert_bm25_stats.json", "r") as f:
    stats = json.load(f)

In [14]:
print(stats.keys())
colbert_stats = stats["ColBERT"]
bm25_stats = stats["BM25"]

dict_keys(['ColBERT', 'BM25'])

In [15]:
# Use rich to print a table for ColBERT vs. BM25
from rich import print
from rich.table import Table

table = Table(title="ColBERT vs. BM25")
table.add_column("Metric")
table.add_column("ColBERT")
table.add_column("BM25")

for metric in colbert_stats[0].keys():
    colbert_values = [stats[metric] for stats in colbert_stats]
    bm25_values = [stats[metric] for stats in bm25_stats]
    
    table.add_row(metric, f"{np.mean(colbert_values):.4f}", f"{np.mean(bm25_values):.4f}")
    
print(table)

        ColBERT vs. BM25        
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ Metric    ┃ ColBERT ┃ BM25   ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ MRR       │ 0.9825  │ 0.7442 │
│ nDCG      │ 0.9699  │ 0.7141 │
│ Precision │ 0.7040  │ 0.5220 │
│ Recall    │ 0.7021  │ 0.2086 │
└───────────┴─────────┴────────┘

In [18]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever,
)

index_name = keyspace

In [19]:
# initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [20]:
index = pc.Index(index_name)

In [21]:
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings()

In [22]:
from pinecone_text.sparse import BM25Encoder

# or from pinecone_text.sparse import SpladeEncoder if you wish to work with SPLADE

# use default tf-idf values
bm25_encoder = BM25Encoder().default()

In [25]:
bm25_encoder.fit(all_texts)

bm25_encoder.dump(cwd + "/bm25_encoder.json")

  0%|          | 0/9992 [00:00<?, ?it/s]

In [28]:
pinecone_retriever = PineconeHybridSearchRetriever(
    embeddings=openai_embeddings, sparse_encoder=bm25_encoder, index=index
)

In [29]:
result = pinecone_retriever.invoke("What is the capital of France?", k=5)

In [31]:
print(result[:2])

[
    Document(
        page_content='France has been one of the most dominant agricultural centers of Europe for centuries. That 
gave France an important role in European and, to some extent world, affairs in the pre-industrial age.he major 
agricultural products that place France among the top producers in the world market are sugar beets, wine, milk, 
beef and veal, cereals, and oilseeds.',
        metadata={'row_id': 108.0}
    ),
    Document(
        page_content="The most important crop was wheat. Wheat was used to make flour. The farmers ground the wheat
grains in the seigneur's flour mill. They would pay the seigneur with part of the … flour for the use of his mill. 
France may be a small country in Western Europe, but the impact of French cu…. 2  The Weather in France Though 
France is only the size of Texas, it is situated in an area where weather varies greatly.",
        metadata={'row_id': 108.0}
    )
]

In [32]:
from tqdm import tqdm

pinecone_retrieved_list = []
pinecone_relevant_list = []
for row in tqdm(subset):
    query = row['query']
    n_results = len(row['passages']['is_selected'])
    raw_results = pinecone_retriever.invoke(query, k=n_results)
    retrieved_list = [result.page_content for result in raw_results]
    relevant_list = row['passages']['passage_text']

    pinecone_retrieved_list.append(retrieved_list)
    pinecone_relevant_list.append(relevant_list)

100%|██████████| 1000/1000 [03:41<00:00,  4.51it/s]


In [34]:
# calculate stats
pinecone_stats = [compute_all_stats(retrieved_list, relevant_list) for retrieved_list, relevant_list in
                  zip(pinecone_retrieved_list, pinecone_relevant_list)] 

In [36]:
# Print a table

table = Table(title="ColBERT vs. BM25 vs. Pinecone Hybrid")

table.add_column("Metric")
table.add_column("ColBERT")
table.add_column("BM25")
table.add_column("Pinecone")

for metric in colbert_stats[0].keys():
    colbert_values = [stats[metric] for stats in colbert_stats]
    bm25_values = [stats[metric] for stats in bm25_stats]
    pinecone_values = [stats[metric] for stats in pinecone_stats]
    
    table.add_row(metric, f"{np.mean(colbert_values):.4f}", f"{np.mean(bm25_values):.4f}", f"{np.mean(pinecone_values):.4f}")
    
print(table)

   ColBERT vs. BM25 vs. Pinecone Hybrid    
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ Metric    ┃ ColBERT ┃ BM25   ┃ Pinecone ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ MRR       │ 0.9825  │ 0.7442 │ 0.9826   │
│ nDCG      │ 0.9699  │ 0.7141 │ 0.9819   │
│ Precision │ 0.7040  │ 0.5220 │ 0.9393   │
│ Recall    │ 0.7021  │ 0.2086 │ 0.3770   │
└───────────┴─────────┴────────┴──────────┘